<a href="https://colab.research.google.com/github/Tam02112003/Python_co_ban/blob/main/Pyspark_c%C6%A1_b%E1%BA%A3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
SpySpark don gian
"""
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("PySpark Basic Example") \
    .getOrCreate()

# Dữ liệu mẫu
data = [("Alice", 25), ("Bob", 30), ("Charlie", 28)]
columns = ["Name", "Age"]


# Tạo DataFrame
df = spark.createDataFrame(data, columns)

df.show()



+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 28|
+-------+---+



In [2]:
# Lấy schema
df.printSchema()

# Lọc dữ liệu (người trên 26 tuổi)
df.filter(df["Age"] > 26).show()

# Chọn cột
df.select("Name").show()

# Thêm cột mới
from pyspark.sql.functions import col
df = df.withColumn("AgePlus5", col("Age") + 5)
df.show()


root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

+-------+---+
|   Name|Age|
+-------+---+
|    Bob| 30|
|Charlie| 28|
+-------+---+

+-------+
|   Name|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+

+-------+---+--------+
|   Name|Age|AgePlus5|
+-------+---+--------+
|  Alice| 25|      30|
|    Bob| 30|      35|
|Charlie| 28|      33|
+-------+---+--------+



In [3]:
data = [("Alice", "Math", 90), ("Alice", "English", 85),
        ("Bob", "Math", 75), ("Bob", "English", 80)]
columns = ["Name", "Subject", "Score"]
df = spark.createDataFrame(data, columns)

# Trung bình điểm theo tên
df.groupBy("Name").avg("Score").show()

# Đếm số môn mỗi người học
df.groupBy("Name").count().show()


+-----+----------+
| Name|avg(Score)|
+-----+----------+
|Alice|      87.5|
|  Bob|      77.5|
+-----+----------+

+-----+-----+
| Name|count|
+-----+-----+
|Alice|    2|
|  Bob|    2|
+-----+-----+



In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def convert_score(score):
    if score >= 85:
        return "A"
    elif score >= 70:
        return "B"
    else:
        return "C"

grade_udf = udf(convert_score, StringType())

df = df.withColumn("Grade", grade_udf(df["Score"]))
df.show()


+-----+-------+-----+-----+
| Name|Subject|Score|Grade|
+-----+-------+-----+-----+
|Alice|   Math|   90|    A|
|Alice|English|   85|    A|
|  Bob|   Math|   75|    B|
|  Bob|English|   80|    B|
+-----+-------+-----+-----+



In [6]:
# Đọc CSV
df_csv = spark.read.option("header", True).csv("students.csv")
df_csv.show()

# Ghi ra Parquet
df.write.parquet("output_students.parquet")


+---+-------+---+------+-----+
| ID|   Name|Age|Gender|Score|
+---+-------+---+------+-----+
|  1|  Alice| 20|Female|   85|
|  2|    Bob| 22|  Male|   78|
|  3|Charlie| 21|  Male|   90|
|  4|  Diana| 23|Female|   92|
|  5|    Eva| 20|Female|   88|
|  6|  Frank| 24|  Male|   75|
+---+-------+---+------+-----+



In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Dữ liệu mẫu
data = [(1, 1.0), (2, 2.0), (3, 2.9), (4, 4.1), (5, 5.0)]
df = spark.createDataFrame(data, ["x", "y"])

# Chuyển thành vector features
vec = VectorAssembler(inputCols=["x"], outputCol="features")
df = vec.transform(df)

# Huấn luyện mô hình hồi quy tuyến tính
lr = LinearRegression(featuresCol="features", labelCol="y")
model = lr.fit(df)

# Dự đoán
pred = model.transform(df)
pred.select("x", "y", "prediction").show()


+---+---+------------------+
|  x|  y|        prediction|
+---+---+------------------+
|  1|1.0|0.9800000000000015|
|  2|2.0|1.9900000000000009|
|  3|2.9|               3.0|
|  4|4.1|              4.01|
|  5|5.0|              5.02|
+---+---+------------------+

